In [24]:
import os
import torch
import numpy as np
from singleVis.visualizer import DataVisualizer

epoch_start = 1
epoch_end = 50
epoch_period = 1
split = 0
input_dims = 512  # Adjust according to your data
output_dims = 2
units = 256
hidden_layer = 3
n_neighbors = 15
s_n_epochs = 5
b_n_epochs = 5
t_n_epochs = 100
final_k = 0
persistence = 0
DEVICE = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

# Load the model
model_path = "/home/yiming/cophi/projects/fork/backdoor_attack/dynamic/BadNet_MNIST_noise_salt_pepper_s0_t1/Model"
model_file = next(f for f in os.listdir(model_path) if f.endswith('.pth'))
# model_file = "/home/yiming/cophi/projects/fork/backdoor_attack/dynamic/BadNet_MNIST_noise_salt_pepper_s0_t1/visualization/visualize/TimeVis/vismodel/1.pth"

# Initialize the data provider and model
from singleVis.data_provider import DataProvider
from singleVis.visualization_model import SingleVisualizationModel

selected_idxs = list(range(118))

# Data Provider
data_provider = DataProvider(model_path, 1, 50, 1, selected_idxs)

# Initialize model
model = SingleVisualizationModel(
    input_dims=512,
    output_dims=2,
    units=units,
    hidden_layer=hidden_layer,
    device=DEVICE
)
model = model.to(DEVICE)

# Load the model state
checkpoint = torch.load(os.path.join(model_path, model_file))
model.load_state_dict(checkpoint['state_dict'])
model.eval()

# Create instances of Projector and DataVisualizer
visualizer = DataVisualizer(data_provider, model)

# Directories to save embeddings and scales
projection_dir = "/home/yiming/cophi/projects/fork/backdoor_attack/dynamic/BadNet_MNIST_noise_salt_pepper_s0_t1/visualization/visualize/TimeVis/projection"
scale_dir = "/home/yiming/cophi/projects/fork/backdoor_attack/dynamic/BadNet_MNIST_noise_salt_pepper_s0_t1/visualization/visualize/TimeVis/scale"
os.makedirs(projection_dir, exist_ok=True)
os.makedirs(scale_dir, exist_ok=True)

# Iterate over each epoch's train data
for epoch in range(1,51):
    # Get 2D embedding for each sample
    data = data_provider.train_representation(epoch)
    data = torch.from_numpy(data).to(device=model.device, dtype=torch.float)
    embedded = model.encoder(data).cpu().detach().numpy()
    np.save(os.path.join(projection_dir, f"{epoch}.npy"), embedded)

    # Get the scale for the 2D embedding
    x_min, y_min, x_max, y_max = visualizer._get_epoch_plot_measures(epoch)
    scale = [x_min, y_min, x_max, y_max]
    np.save(os.path.join(scale_dir, f"{epoch}.npy"), scale)




In [25]:
# Iterate over each epoch's train data
for epoch in range(1,51):
    # Get 2D embedding for each sample
    data = data_provider.train_representation(epoch)
    data = torch.from_numpy(data).to(device=model.device, dtype=torch.float)
    embedded = model.encoder(data).cpu().detach().numpy()
    # np.save(os.path.join(projection_dir, f"{epoch}.npy"), embedded)
    print(embedded[8])

    # Get the scale for the 2D embedding
    x_min, y_min, x_max, y_max = visualizer._get_epoch_plot_measures(epoch)
    # print(x_min, y_min, x_max, y_max)
    scale = [x_min, y_min, x_max, y_max]
    # np.save(os.path.join(scale_dir, f"{epoch}.npy"), scale)

[-160.89078 -118.19478]
[-161.05972 -118.3102 ]
[-161.7562   -118.834885]
[-162.80093 -119.62056]
[-163.70667 -120.26163]
[-165.6406  -121.64457]
[-158.42764 -116.3234 ]
[-34.005505 -20.249378]
[-11.587313   -3.3875835]
[-8.898812  -2.1628888]
[-7.6629024 -1.3069928]
[-7.221317   -0.83681136]
[-6.968803  -0.5487035]
[-6.882488   -0.37278202]
[-6.8924994  -0.24030332]
[-6.9025855  -0.12342481]
[-6.8941245  -0.01222066]
[-6.9818797   0.05454417]
[-6.965053    0.10792228]
[-6.981767    0.15051813]
[-7.026916    0.20060702]
[-7.042978    0.24368544]
[-7.0820327   0.28806183]
[-7.091808    0.32102177]
[-7.111587    0.35287544]
[-7.12676     0.38139316]
[-7.1454616   0.41037056]
[-7.1514716  0.4347398]
[-7.189914    0.46850654]
[-7.1917605   0.49301407]
[-7.2238803  0.5208127]
[-7.237581    0.54264516]
[-7.2438235  0.5640132]
[-7.2582583  0.5825469]
[-7.270948   0.6005303]
[-7.2753615  0.612951 ]
[-7.288112    0.62778825]
[-7.2926574  0.6392638]
[-7.296657    0.64907616]
[-7.3032317  0.65712

In [26]:
import numpy as np
import os

projection_dir = "/home/yiming/cophi/projects/fork/backdoor_attack/dynamic/BadNet_MNIST_noise_salt_pepper_s0_t1/visualization/visualize/TimeVis/projection"

# Iterate over each file in the projection directory
for file_name in os.listdir(projection_dir):
    if file_name.endswith('.npy'):
        file_path = os.path.join(projection_dir, file_name)
        
        # Load the embedding data
        embedding_data = np.load(file_path)
        
        # Select the first 117 rows
        selected_data = embedding_data[:117]
        
        # Save the selected data back to the file
        np.save(file_path, selected_data)


In [27]:
import numpy as np
import os

projection_dir = "/home/yiming/cophi/projects/fork/backdoor_attack/dynamic/BadNet_MNIST_noise_salt_pepper_s0_t1/visualization/visualize/TimeVis/scale"

# Iterate over each file in the projection directory
for file_name in os.listdir(projection_dir):
    if file_name.endswith('.npy'):
        file_path = os.path.join(projection_dir, file_name)
        
        # Load the embedding data
        embedding_data = np.load(file_path)
        
        # Extract the numeric part of the file name and add 25
        file_index = int(file_name.split('.')[0]) + 25
        
        # Save the data to a new file with the updated index
        new_file_path = os.path.join(projection_dir, f"{file_index}.npy")
        np.save(new_file_path, embedding_data)